поэтому надо использовать лок, чтобы эту гонку избежать
вот тут будет без лока

In [1]:
import concurrent.futures
import time


class BankAccount:
    def __init__(self):
        self.balance = 100  # тот ресурс, котоый будет использован потоками


    def update(self, transaction, amount):
        print(f'{transaction} started')

        tmp_balance = self.balance
        tmp_balance += amount
        time.sleep(1)  # вызывает переключение контекста
        self.balance = tmp_balance

        print(f'\n{transaction} ended')


if __name__ == '__main__':
    acc = BankAccount()
    print(f'main started, acc balance = {acc.balance}')

    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as ex:
        ex.map(acc.update, ('refill', 'withdraw'), (100, -190))

    print(f'end of main, balance = {acc.balance}')


main started, acc balance = 100
refill started
withdraw started

withdraw ended
refill ended

end of main, balance = 200


мы создаем некий объект, он мб в состоянии locked и unlocked
чтобы захватить объект, вызываем ф-ю acquire
чтобы освободить - ф-ю release

--------------------
а вот далее уже будет с лок
короче,какая тема, там в конструкторе есть строка self.lock_obj = threading.Lock(),и есть блок with, 
в самом конструкторе ничего не блокируется сразу, однако когда запускается блок with, там уже происходит блокировка,
также там обе функции acquire и release вызываются автоматически, 
пока один поток не зайдет и не сделает свои дела, другой не попадет туда

то есть в какой-то степени код будет исполняться последовательно

In [ ]:
import concurrent.futures
import threading
import time


class BankAccount:
    def __init__(self):
        self.balance = 100  # тот ресурс, котоый будет использован потоками
        self.lock_obj = threading.Lock()  # тут мы ничего не захватываем
    def update(self, transaction, amount):
        print(f'{transaction} started')

        with self.lock_obj:  # и вот эта тема уже решает проблему, он захватывает один поток, а другой уже ждет
            tmp_balance = self.balance
            tmp_balance += amount
            time.sleep(1)  # вызывает переключение контекста
            self.balance = tmp_balance

        print(f'\n{transaction} ended')


if __name__ == '__main__':


    acc = BankAccount()
    print(f'main started, acc balance = {acc.balance}')
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as ex:
        ex.map(acc.update, ('refill', 'withdraw'), (100, -190))
    print(f'end of main, balance = {acc.balance}')


In [ ]:
блок with можно не использовать, а делать так:
    self.lock_obj.acquire()
    tmp_balance = self.balance
    tmp_balance += amount
    time.sleep(1)  # вызывает переключение контекста
    self.balance = tmp_balance
    self.lock_obj.release()